### Import essential libraries

In [198]:
import pandas as pd
import numpy as np
import texthero as hero # cleaning pandas columns
import time
import nltk
import spacy # check similarity of words precisely
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
nlp = spacy.load('en_core_web_lg')

### Read `praise` and `tag` data into pandas dataframe and perform some cleansing

In [199]:
df_praise = pd.read_csv("data/sample_praise.csv")
df_tag = pd.read_csv("data/tags.csv")
df_praise.iloc[:,5] = hero.clean(df_praise.iloc[:,5])

/home/shawn/.virtualenvs/LTF/lib/python3.8/site-packages/texthero/preprocessing.py:46: FutureWarning: The default value of regex will change from True to False in a future version.
  return input.str.replace(r"^\d+\s|\s\d+\s|\s\d+$", " ")
/home/shawn/.virtualenvs/LTF/lib/python3.8/site-packages/texthero/preprocessing.py:81: FutureWarning: The default value of regex will change from True to False in a future version.
  return input.str.replace(pat, '')


Drop unecessary columns.

In [200]:
df_praise = pd.concat([df_praise.iloc[:,0], df_praise.iloc[:,3:15]], axis=1)

In [201]:
df_praise.head(5)

,Category Code,To,From,Reason for dishing,Server,Date,Room,v1 norm,v2 norm,v3 norm,Avg %,IH per Praise,IH per person
0,TEC15,liviade,cranders71,always caring finicky bot valued community members,Telegram,2021-01-13,TE Praise,30,10.0,40.0,6.549.137.185,4.911.852.889,6.767.403.323
1,TEC12,chuygarcia92,Tam2140,masana temples recommendation nice mix low high beats,Token Engineering Commons,2021-04-30,🙏praise,1000,100.0,100.0,723.394.157,8.680.729.884,718.127.319
2,TEC12,iviangita,iviangita,joining legal weekly sync,Token Engineering Commons,2021-04-30,🙏praise,1000,80.0,100.0,6.595.463.701,7.914.556.442,2.387.003.435
3,TEC12,ygganderson,JessicaZartler,presence energy smiles comms working group today,Telegram,2020-11-24,TE Praise,20,60.0,30.0,1.283.215.058,NaN,NaN
4,TEC12,metaverde,iviangita,mentioning retweeting te commons socials past week thank helping us grow token engineering commons community spreading message,Telegram,2021-01-22,TE Commons,NaN,NaN,NaN,0,198.159.943.382.873,3.170.559.094


In [219]:
# reading praise data
df_praise = pd.read_csv("data/cleaned-non-quantifier-data.csv", index_col=False)

In [222]:
praise_set = hero.clean(df_praise['Reason for dishing'])

/home/shawn/.virtualenvs/LTF/lib/python3.8/site-packages/texthero/preprocessing.py:46: FutureWarning: The default value of regex will change from True to False in a future version.
  return input.str.replace(r"^\d+\s|\s\d+\s|\s\d+$", " ")
/home/shawn/.virtualenvs/LTF/lib/python3.8/site-packages/texthero/preprocessing.py:81: FutureWarning: The default value of regex will change from True to False in a future version.
  return input.str.replace(pat, '')


In [223]:
df_praise_set = pd.DataFrame(praise_set).reset_index()
df_praise_set.columns = [0,1]

In [224]:
df_praise_set

,0,1
0,0,hosting kicking params party
1,1,hosting leading lot params parties
2,2,testing deploying bot record display meetings
3,3,huge success mvv process
4,4,awesome work recorder bot help documentation always getting recordings going pushing transparency wg good personality
...,...,...
7620,7620,presence great contributions soft gov wg sync
7621,7621,participating praisegiving call
7622,7622,sharing gitcoin blog post lots great info https twitter com owocki status
7623,7623,joining te praise channel tec lab today


In [225]:
df_tag

,tag,keyword
0,TEC1,comms article blog organized presentation graphic design website marketing seo social platform discord telegram forum discourse medium linkedin website article retweeting retweet retweets organized presentation communications
1,TEC2,culture build soft gov survey vote voting debate
2,TEC3,params parameters param parties party
3,TEC4,legal strategy
4,TEC5,commons swarm tech dev dapp tech dapp app bug
5,TEC6,hatch outreach onboarding onboard outreach
6,TEC7,omega survey philosophy
7,TEC8,stewards github steward project management
8,TEC9,labs lab
9,TEC10,transparency youtube recording record


In [226]:
from typing import Callable

def spacy_similarity(doc1, doc2):
    return nlp(doc1).similarity(nlp(doc2))

def max_intersection_similarity(doc1, doc2):
    doc1 = set(doc1.split(" "))
    doc2 = set(doc2.split(" "))
    similarity = len(doc1.intersection(doc2))# / len(doc1.union(doc2))
    return similarity

def jaccard_similarity(doc1, doc2):
    doc1 = set(doc1.split(" "))
    doc2 = set(doc2.split(" "))
    similarity = len(doc1.intersection(doc2)) / len(doc1.union(doc2))
    return similarity

def jaccard_label(praise_reason: str, sim_function: Callable):
    tags['similarity'] = tags[1].apply(lambda x: sim_function(x, praise_reason))
    max_tag = tags.sort_values('similarity',ascending=False).iloc[0]
    sim_score = max_tag['similarity']
    max_tag = max_tag[0]
    if sim_score == 0:
        max_tag = np.nan
    return max_tag

In [ ]:
df_praise_set['max_intersection_similarity'] = df_praise_set[1].apply(lambda x: jaccard_label(x, max_intersection_similarity))

In [230]:
df_praise_set['jaccard_similarity'] = df_praise_set[1].apply(lambda x: jaccard_label(x, jaccard_similarity))

In [231]:
df_praise_set['spacy_similarity'] = df_praise_set[1].apply(lambda x: jaccard_label(x, spacy_similarity))

<ipython-input-226-17cb5292c381>:4: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return nlp(doc1).similarity(nlp(doc2))


In [232]:
df_praise_set

,0,1,max_intersection_similarity,jaccard_similarity,spacy_similarity
0,0,hosting kicking params party,TEC3,TEC3,TEC3
1,1,hosting leading lot params parties,TEC3,TEC3,TEC3
2,2,testing deploying bot record display meetings,TEC10,TEC10,TEC10
3,3,huge success mvv process,NaN,NaN,TEC2
4,4,awesome work recorder bot help documentation always getting recordings going pushing transparency wg good personality,TEC10,TEC10,TEC10
...,...,...,...,...,...
7620,7620,presence great contributions soft gov wg sync,TEC2,TEC2,TEC2
7621,7621,participating praisegiving call,NaN,NaN,TEC2
7622,7622,sharing gitcoin blog post lots great info https twitter com owocki status,TEC1,TEC1,TEC1
7623,7623,joining te praise channel tec lab today,TEC9,TEC9,TEC15


In [233]:
df_praise_set.to_csv('outputs/similarity_tagging.csv',index=False)

## Below this point is just workspace

In [ ]:

    

praise = pd.DataFrame(praise_set)

#praise['spacy_similarity'] = praise[1].apply(lambda x: jaccard_label(x, spacy_similarity))
praise['max_intersection_similarity'] = praise[1].apply(lambda x: jaccard_label(x, max_intersection_similarity))
#praise['jaccard_similarity'] = praise[1].apply(lambda x: jaccard_label(x, jaccard_similarity))

praise

### `check_freq` function returns most frequent tag for each row

In [7]:
def check_freq(tag_list):
    final_tag_list = []
    try:
        for lst in tag_list:
            str_ = ''
            for item in tag_list:
                str_ = str_ +" "+ item
            moby_tokens = nltk.word_tokenize(str_)
            text = nltk.Text(moby_tokens)

            dist = nltk.FreqDist(text)
            freq_lst = [k for k, v in sorted(dist.items(), key=lambda i: i[1], reverse=True)]
            final_tag_list.append(freq_lst[0])
        return final_tag_list[0]
    except:
        return []

### Iterating through all rows and tag data just for equal words(100% similarity based on tag dataset)

In [8]:
tag_dict = set()
for index_, row in df_tag.iterrows():
    tag_dict.add((row[0], row[1]))
    
tag_dict

{('TEC1',
  'comms article blog organized presentation graphic design website marketing seo social platform discord telegram forum discourse medium linkedin website article retweeting retweet retweets organized presentation communications'),
 ('TEC10', 'transparency youtube recording record'),
 ('TEC12', 'participation retweets'),
 ('TEC13', 'fundraising donating fundraising'),
 ('TEC14', 'technical infrastructure'),
 ('TEC15', 'tec community building'),
 ('TEC16', 'personal praise'),
 ('TEC17', 'gravity conflict nonviolent communication'),
 ('TEC2', 'culture build soft gov survey vote voting debate'),
 ('TEC3', 'params parameters param parties party'),
 ('TEC4', 'legal strategy'),
 ('TEC5', 'commons swarm tech dev dapp tech dapp app bug'),
 ('TEC6', 'hatch outreach onboarding onboard outreach'),
 ('TEC7', 'omega survey philosophy'),
 ('TEC8', 'stewards github steward project management'),
 ('TEC9', 'labs lab')}

In [9]:
praise_set = set()
for index_, row in df_praise.iloc[:100,:].iterrows():
    praise_set.add((index_, row[3]))
    
praise_set

{(0, 'always caring finicky bot valued community members'),
 (1, 'masana temples recommendation nice mix low high beats'),
 (2, 'joining legal weekly sync'),
 (3, 'presence energy smiles comms working group today'),
 (4,
  'mentioning retweeting te commons socials past week thank helping us grow token engineering commons community spreading message'),
 (5,
  'engaging discussion te commons forum https forum tecommons org past week thank helping token engineering commons community share learn'),
 (6,
  'mentioning retweeting te commons socials past week thank helping us grow token engineering commons community spreading message'),
 (7, 'insightful well formed presentation gravity today'),
 (8, 'conducting giving feedback audit'),
 (9, 'last session gravity incredible whole ecosystem'),
 (10,
  'joining hack session staying great discussion anarchy vs community governance'),
 (11, 'bringing board zenhub team'),
 (12, 'joining tec community call https www youtube com watch v dhnrqaow3ic')

In [25]:
tag_dict

{('TEC1',
  'comms article blog organized presentation graphic design website marketing seo social platform discord telegram forum discourse medium linkedin website article retweeting retweet retweets organized presentation communications'),
 ('TEC10', 'transparency youtube recording record'),
 ('TEC12', 'participation retweets'),
 ('TEC13', 'fundraising donating fundraising'),
 ('TEC14', 'technical infrastructure'),
 ('TEC15', 'tec community building'),
 ('TEC16', 'personal praise'),
 ('TEC17', 'gravity conflict nonviolent communication'),
 ('TEC2', 'culture build soft gov survey vote voting debate'),
 ('TEC3', 'params parameters param parties party'),
 ('TEC4', 'legal strategy'),
 ('TEC5', 'commons swarm tech dev dapp tech dapp app bug'),
 ('TEC6', 'hatch outreach onboarding onboard outreach'),
 ('TEC7', 'omega survey philosophy'),
 ('TEC8', 'stewards github steward project management'),
 ('TEC9', 'labs lab')}

In [28]:
a = {1,2,3}

In [29]:
import itertools

In [58]:
sub_praise_set = set(itertools.islice(praise_set, 5))

In [59]:
sub_tag_dict = set(itertools.islice(tag_dict, 3))

In [117]:
list_tag = []
temp_1 = []
counter = 0

start_time = time.time()
for index_praise in praise_set:
    temp_1 = [row_tag[0] for row_tag in tag_dict for item_praise in set(index_praise[1].split(" ")) for item_tag in set(row_tag[1].split(" ")) if nlp(item_praise).similarity(nlp(item_tag)) == 1]
    if len(temp_1)>=1:
        list_tag.append((index_praise[0], check_freq(temp_1)))
        temp_1 = []
    else:
        list_tag.append((index_praise[0],np.nan))
    print(f"row {counter} is done.", end='\r')
    counter+=1

end_time = time.time()
print(f"\nTotal run time for {len(praise_set)} rows is: {end_time - start_time}")

<ipython-input-117-071be6b5bcf1>:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  temp_1 = [row_tag[0] for row_tag in tag_dict for item_praise in set(index_praise[1].split(" ")) for item_tag in set(row_tag[1].split(" ")) if nlp(item_praise).similarity(nlp(item_tag)) == 1]


row 99 is done.
Total run time for 100 rows is: 404.6406993865967


In [119]:
output = pd.DataFrame(list_tag)

In [135]:
praise = pd.DataFrame(praise_set)
praise

,0,1
0,97,putting heads finding bug
1,6,mentioning retweeting te commons socials past week thank helping us grow token engineering commons community spreading message
2,1,masana temples recommendation nice mix low high beats
3,48,contributing latest tec praise quant blog
4,57,joining commons stack simulator sync continuing push project finish line
...,...,...
95,60,starting cadcad bootcamp study group
96,51,tackling remaining issues swiss membership dapp
97,46,visible community work hatch super cool
98,2,joining legal weekly sync


In [107]:
tags = pd.DataFrame(tag_dict)
tags

,0,1
0,TEC2,culture build soft gov survey vote voting debate
1,TEC14,technical infrastructure
2,TEC5,commons swarm tech dev dapp tech dapp app bug
3,TEC10,transparency youtube recording record
4,TEC3,params parameters param parties party
5,TEC7,omega survey philosophy
6,TEC9,labs lab
7,TEC4,legal strategy
8,TEC17,gravity conflict nonviolent communication
9,TEC1,comms article blog organized presentation graphic design website marketing seo social platform discord telegram forum discourse medium linkedin website article retweeting retweet retweets organized presentation communications


In [177]:
praise['original_label'] = output[1]

In [178]:
praise = praise.dropna()

In [179]:
praise['accuracy'] = praise['label'] == praise['original_label']

In [180]:
praise['accuracy'].sum() / len(praise)

0.4142857142857143

In [181]:
praise

,0,1,label,original_label,accuracy
0,97,putting heads finding bug,TEC5,TEC5,True
1,6,mentioning retweeting te commons socials past week thank helping us grow token engineering commons community spreading message,TEC1,TEC5,False
3,48,contributing latest tec praise quant blog,TEC1,TEC1,True
4,57,joining commons stack simulator sync continuing push project finish line,TEC2,TEC5,False
5,4,mentioning retweeting te commons socials past week thank helping us grow token engineering commons community spreading message,TEC1,TEC5,False
...,...,...,...,...,...
93,78,mentioning retweeting commons stack socials past week thank helping us grow commons stack community spreading message,TEC1,TEC5,False
94,98,joining tec weekly sync https www youtube com watch v efhuvu oc0,TEC10,TEC10,True
96,51,tackling remaining issues swiss membership dapp,TEC2,TEC5,False
97,46,visible community work hatch super cool,TEC15,TEC6,False


In [116]:
list_tag

[(97, 'TEC5'), (6, 'TEC5'), (48, nan), (1, nan), (57, 'TEC5')]

In [63]:
['TEC5']

['TEC5']

In [84]:
pd.DataFrame(list_tag)

,0,1
0,97,TEC5
1,6,TEC5
2,48,NaN
3,1,NaN
4,57,TEC5


In [65]:
df_temp = pd.DataFrame(list_tag, columns=['index_', 'tag'])
df_temp

,index_,tag
0,97,TEC5
1,6,TEC5
2,48,NaN
3,1,NaN
4,57,TEC5


In [12]:
df_sample_100 = pd.merge(df_praise, df_temp, how="left", left_index=True, right_on='index_')

In [13]:
df_sample_100 = df_sample_100.loc[df_sample_100.index.dropna()]

In [14]:
df_sample_100.columns

Index(['Category Code', 'To', 'From', 'Reason for dishing', 'Server', 'Date',
       'Room', 'v1 norm', 'v2 norm', 'v3 norm', 'Avg %', 'IH per Praise',
       'IH per person', 'index_', 'tag'],
      dtype='object')

In [15]:
df_compare = df_sample_100[['Category Code', 'tag', 'Reason for dishing']]

In [16]:
df_compare.isnull().sum()

Category Code          0
tag                   30
Reason for dishing     0
dtype: int64

In [26]:
df_compare.sort_index()

,Category Code,tag,Reason for dishing
0.0,TEC14,TEC5,putting heads finding bug
1.0,TEC12,TEC5,mentioning retweeting te commons socials past week thank helping us grow token engineering commons community spreading message
2.0,TEC12,NaN,masana temples recommendation nice mix low high beats
3.0,TEC10,TEC1,contributing latest tec praise quant blog
4.0,TEC12,TEC5,joining commons stack simulator sync continuing push project finish line
...,...,...,...
95.0,TEC5,NaN,starting cadcad bootcamp study group
96.0,TEC5,TEC5,tackling remaining issues swiss membership dapp
97.0,TEC5,TEC6,visible community work hatch super cool
98.0,TEC12,TEC4,joining legal weekly sync


In [ ]:
n praise records
m tags

n_i has p_i words
m_j has q_j words

so total complexity is

for each i in n
    for each j in m
     for each p_i_k in p_i
        for each q_j_l in q_j
            sims.append(compare_similarity(p_i_k, q_j_l))
        list_of_tags.append(most_frequent(sim))
        sims = []
    

Start with a praise data set. For each praise record, we will compare the similarity to each tag label.

Give the first row of praise data, split the 'reason for praise' into a list of words.
Now go through each

In [18]:
# list_tag = []
# temp_1 = []
# temp_2 = []
# temp_3 = []
# counter = 0

# start_time = time.time()
# for index_praise in praise_set:
#     for row_tag in tag_dict:
#         temp_set_1 = set(index_praise[1].split(" "))
#         for item_praise in temp_set_1:
#             temp_set_2 = set(row_tag[1].split(" "))
#             for item_tag in temp_set_2:
#                 tag_token = nlp(item_tag)
#                 reason_token = nlp(item_praise)
#                 if reason_token.similarity(tag_token) == 1:
#                     temp_1.append(row_tag[0])
#             if len(temp_1)>=1:
#                 temp_2.append(check_freq(temp_1))
#                 temp_1 = []
#         if len(temp_2)>=1:
#             temp_3.append(check_freq(temp_2))
#             temp_2 = []
#     if len(temp_3)>=1:
#         list_tag.append((index_praise[0], check_freq(temp_3)))
#         temp_3 = []
#     else:
#         list_tag.append((index_praise[0],np.nan))
#     print(f"row {counter} is done.", end='\r')
#     counter+=1

# end_time = time.time()
# print(f"\nTotal run time for {df_praise.shape[0]} rows is: {end_time - start_time}")

In [19]:
# for i in range(1, 101):
#     print('#'*i+' '*(101-i)+f'{i}%', end='\r')
#     time.sleep(0.1)